### Load Registered Model, Run Inference on Test Data

In [0]:
import pandas as pd
import joblib

# load objects saved in prior notebook
X_test = pd.read_parquet('/dbfs/tmp/X_train.parquet')
X_val = pd.read_parquet('/dbfs/tmp/X_val.parquet')
y_test = pd.read_parquet('/dbfs/tmp/y_train.parquet')
y_val = pd.read_parquet('/dbfs/tmp/y_val.parquet')
# best_model = joblib.load('/dbfs/tmp/best_model.pkl') # we don't need this since we're loading the model from MLflow registry

In [0]:
import numpy as np 
import mlflow
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, root_mean_squared_error

# load registered model
model_name = 'xgb_housing_model'
model_version = 'latest'
model_uri = f'models:/{model_name}/{model_version}'
loaded_model = mlflow.xgboost.load_model(model_uri)

# run inference on test data
y_test_pred = loaded_model.predict(X_test)

# calculate test metrics 
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

print(f'Test RMSE: {test_rmse}')
print(f'Test R2: {test_r2}')
print(f'Test MAPE: {test_mape}')

/databricks/python/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1711007984306980>, line 12
      9 loaded_model = mlflow.xgboost.load_model(model_uri)
     11 # run inference on test data
---> 12 y_test_pred = loaded_model.predict(X_test)
     14 # calculate test metrics 
     15 test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

File /databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:580, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    578     patch_function.call(call_original, *args, **kwargs)
    579 else:
--> 580     patch_function(call_original, *args, **kwargs)
    582 session.state = "succeeded"
    584 try_log_autologging_event(
    585     AutologgingEventLogger.get_logger().log_patch_function_success,
    586     session,
   (...)
    590     kwargs,
    591 )

File /databricks/python/lib/python3.12/site-package

In [0]:
# set up model monitoring with EvidentlyAI
%pip install evidently
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently.report import Report

from evidently import metrics
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import RegressionPerformanceMetrics

from evidently.test_suite import TestSuite
from evidently.tests import *
from evidently.test_preset import DataDriftTestPreset
from evidently.tests.base_test import TestResult, TestStatus

from evidently.ui.dashboards import DashboardPanelPlot
from evidently.ui.dashboards import DashboardPanelTestSuite
from evidently.ui.dashboards import PanelValue
from evidently.ui.dashboards import PlotType
from evidently.ui.dashboards import ReportFilter
from evidently.ui.dashboards import TestFilter
from evidently.ui.dashboards import TestSuitePanelType
from evidently.renderers.html_widgets import WidgetSize

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1711007984306980>, line 12
      9 loaded_model = mlflow.xgboost.load_model(model_uri)
     11 # run inference on test data
---> 12 y_test_pred = loaded_model.predict(X_test)
     14 # calculate test metrics 
     15 test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

File /databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:580, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    578     patch_function.call(call_original, *args, **kwargs)
    579 else:
--> 580     patch_function(call_original, *args, **kwargs)
    582 session.state = "succeeded"
    584 try_log_autologging_event(
    585     AutologgingEventLogger.get_logger().log_patch_function_success,
    586     session,
   (...)
    590     kwargs,
    591 )

File /databricks/python/lib/python3.12/site-package

In [0]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1711007984306980>, line 12
      9 loaded_model = mlflow.xgboost.load_model(model_uri)
     11 # run inference on test data
---> 12 y_test_pred = loaded_model.predict(X_test)
     14 # calculate test metrics 
     15 test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

File /databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:580, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    578     patch_function.call(call_original, *args, **kwargs)
    579 else:
--> 580     patch_function(call_original, *args, **kwargs)
    582 session.state = "succeeded"
    584 try_log_autologging_event(
    585     AutologgingEventLogger.get_logger().log_patch_function_success,
    586     session,
   (...)
    590     kwargs,
    591 )

File /databricks/python/lib/python3.12/site-package

In [0]:
reference_data = X_val.copy() # start with val data
reference_data['target'] = y_val # add actual target values
reference_data['prediction'] = loaded_model.predict(X_val) # add model predictions

current_data = X_test.copy() # start with test features
current_data['target'] = y_test # add actual target values
current_data['prediction'] = loaded_model.predict(X_test) # add model predictions to the test data

# create a report with both Data Drift and Performance Monitoring
report = Report(metrics=[
    DataDriftPreset(),
    RegressionPerformanceMetrics()
])

# run the report
report.run(reference_data=reference_data, current_data=current_data)
report

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1711007984306980>, line 12
      9 loaded_model = mlflow.xgboost.load_model(model_uri)
     11 # run inference on test data
---> 12 y_test_pred = loaded_model.predict(X_test)
     14 # calculate test metrics 
     15 test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

File /databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:580, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    578     patch_function.call(call_original, *args, **kwargs)
    579 else:
--> 580     patch_function(call_original, *args, **kwargs)
    582 session.state = "succeeded"
    584 try_log_autologging_event(
    585     AutologgingEventLogger.get_logger().log_patch_function_success,
    586     session,
   (...)
    590     kwargs,
    591 )

File /databricks/python/lib/python3.12/site-package

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(current_data['target'], current_data['prediction'], alpha=0.5)
plt.plot([current_data['target'].min(), current_data['target'].max()], 
         [current_data['target'].min(), current_data['target'].max()], 
         color='red', linestyle='--', linewidth=2)
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.title('Actual vs Predicted Target')
plt.grid(True)
plt.show()

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1711007984306980>, line 12
      9 loaded_model = mlflow.xgboost.load_model(model_uri)
     11 # run inference on test data
---> 12 y_test_pred = loaded_model.predict(X_test)
     14 # calculate test metrics 
     15 test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

File /databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:580, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    578     patch_function.call(call_original, *args, **kwargs)
    579 else:
--> 580     patch_function(call_original, *args, **kwargs)
    582 session.state = "succeeded"
    584 try_log_autologging_event(
    585     AutologgingEventLogger.get_logger().log_patch_function_success,
    586     session,
   (...)
    590     kwargs,
    591 )

File /databricks/python/lib/python3.12/site-package

### Modify the Data to Ensure Model Monitoring Works

In [0]:
# modify data (2 features) according to assignment prompt
current_data['rooms_per_household'] = current_data['rooms_per_household']*1.2
current_data['median_income'] = current_data['median_income']*1.2

# prepare a dataframe that we can make predictions on
X_test_current = current_data.drop(columns=['target', 'prediction'])

# re-run predictions on modified current_data
current_data['prediction'] = loaded_model.predict(X_test_current) # make predictions on modified current_data and add to dataframe

# run the report
report.run(reference_data=reference_data, current_data=current_data)
report

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1711007984306980>, line 12
      9 loaded_model = mlflow.xgboost.load_model(model_uri)
     11 # run inference on test data
---> 12 y_test_pred = loaded_model.predict(X_test)
     14 # calculate test metrics 
     15 test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

File /databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:580, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    578     patch_function.call(call_original, *args, **kwargs)
    579 else:
--> 580     patch_function(call_original, *args, **kwargs)
    582 session.state = "succeeded"
    584 try_log_autologging_event(
    585     AutologgingEventLogger.get_logger().log_patch_function_success,
    586     session,
   (...)
    590     kwargs,
    591 )

File /databricks/python/lib/python3.12/site-package

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(current_data['target'], current_data['prediction'], alpha=0.5)
plt.plot([current_data['target'].min(), current_data['target'].max()], 
         [current_data['target'].min(), current_data['target'].max()], 
         color='red', linestyle='--', linewidth=2)
plt.xlabel('Actual Target')
plt.ylabel('Predicted Target')
plt.title('Actual vs Predicted Target')
plt.grid(True)
plt.show()

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1711007984306980>, line 12
      9 loaded_model = mlflow.xgboost.load_model(model_uri)
     11 # run inference on test data
---> 12 y_test_pred = loaded_model.predict(X_test)
     14 # calculate test metrics 
     15 test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

File /databricks/python/lib/python3.12/site-packages/mlflow/utils/autologging_utils/safety.py:580, in safe_patch.<locals>.safe_patch_function(*args, **kwargs)
    578     patch_function.call(call_original, *args, **kwargs)
    579 else:
--> 580     patch_function(call_original, *args, **kwargs)
    582 session.state = "succeeded"
    584 try_log_autologging_event(
    585     AutologgingEventLogger.get_logger().log_patch_function_success,
    586     session,
   (...)
    590     kwargs,
    591 )

File /databricks/python/lib/python3.12/site-package